# Capstone - Push data Chunk
### 2nd Survey **(Student Service Awareness)**

## Importing & Authenticating

In [80]:
from google.colab import drive
drive.mount('/content/drive')

import google.colab
google.colab.auth.authenticate_user()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
import gspread
import pandas as pd
import pandas_gbq

from google.oauth2 import service_account
from google.cloud import bigquery

In [82]:
# authenticate with Google using the service account credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/.cred/cred.json', scopes=scope)
client = gspread.authorize(creds)

# open the sheet by its title
sheet = client.open('Student_Survey_2').sheet1

# get all values from the sheet
values = sheet.get_all_values()

# convert values to a Pandas dataframe
df2 = pd.DataFrame(values[1:], columns=values[0])


In [83]:
len(df2)

136

In [84]:
df2.head(3)

,Timestamp,In which cohort (Intake) did you start the DAB program?,Which semester are you currently enrolled in?,Are the events and services of the college effectively communicated?,Which of the following services provided by the SRC/College are you interested in exploring or hearing more about?,"According to you, are there sufficient transportation service options available between Downtown and South Campus during events?","In your opinion, would it be beneficial for the students to have additional events in Downtown, such as job fairs, cultural events, games, and career services?",Do you have any suggestions or feedback regarding the services and events offered by the college?,Is the duration of breaks between lectures appropriate?,"How would you rate the study areas currently available in or near the Downtown campus, on a scale of 1 to 5?",Would you like to be in the same section throughout the duration of the program?
0,3/26/2023 18:40:55,Winter 2022,4th Semester,No,Commute/Transportation,No,Yes,,No,Yes,No
1,3/27/2023 18:23:42,Winter 2022,4th Semester,No,"Career Services, Food Bank, SRC Service Office",No,Yes,,No,2,No
2,3/28/2023 10:05:32,Winter 2023,1st Semester,No,"Housing, Career Services, Food Bank, Commute/T...",No,Yes,,Yes,3,Yes


In [85]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 11 columns):
 #   Column                                                                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                                                                           --------------  ----- 
 0   Timestamp                                                                                                                                                        136 non-null    object
 1   In which cohort (Intake) did you start the DAB program?                                                                                                          136 non-null    object
 2   Which semester are you currently enrolled in?                                                                                         

## Creating a DataFrame

In [86]:
df2.columns

Index(['Timestamp', 'In which cohort (Intake) did you start the DAB program?',
       'Which semester are you currently enrolled in?',
       'Are the events and services of the college effectively communicated?',
       'Which of the following services provided by the SRC/College are you interested in exploring or hearing more about?',
       'According to you, are there sufficient transportation service options available between Downtown and South Campus during events?',
       'In your opinion, would it be beneficial for the students to have additional events in Downtown, such as job fairs, cultural events, games, and career services?',
       'Do you have any suggestions or feedback regarding the services and events offered by the college?',
       'Is the duration of breaks between lectures appropriate?',
       'How would you rate the study areas currently available in or near the Downtown campus, on a scale of 1 to 5?',
       'Would you like to be in the same section throughout

In [87]:
#Renaming it columns to a particular format
df2 = df2.rename(columns={
    'In which cohort (Intake) did you start the DAB program?': 'Intake',
    'Which semester are you currently enrolled in?': 'Semester',
    'Are the events and services of the college effectively communicated?': 'Services_comm',
    'Which of the following services provided by the SRC/College are you interested in exploring or hearing more about?': 'Services_list',
    'According to you, are there sufficient transportation service options available between Downtown and South Campus during events?': 'Commute',
    'In your opinion, would it be beneficial for the students to have additional events in Downtown, such as job fairs, cultural events, games, and career services?': 'Additional_events',
    'Do you have any suggestions or feedback regarding the services and events offered by the college?': 'Events_suggestions',
    'Is the duration of breaks between lectures appropriate?': 'Breaks_btwn_Lecture',
    'How would you rate the study areas currently available in or near the Downtown campus, on a scale of 1 to 5?': 'Study_area_rating',
    'Would you like to be in the same section throughout the duration of the program?': 'Same_section',
})



## Data Cleaning

In [88]:
dd = df2.copy()

In [89]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Timestamp            136 non-null    object
 1   Intake               136 non-null    object
 2   Semester             136 non-null    object
 3   Services_comm        136 non-null    object
 4   Services_list        136 non-null    object
 5   Commute              136 non-null    object
 6   Additional_events    136 non-null    object
 7   Events_suggestions   136 non-null    object
 8   Breaks_btwn_Lecture  136 non-null    object
 9   Study_area_rating    136 non-null    object
 10  Same_section         136 non-null    object
dtypes: object(11)
memory usage: 11.8+ KB


In [90]:
#Datatypes are corrected and the unwanted columns are dropped
df2=df2.drop(['Timestamp'],axis=1)

In [91]:
#Checking Null Values
df2.isnull().sum()

Intake                 0
Semester               0
Services_comm          0
Services_list          0
Commute                0
Additional_events      0
Events_suggestions     0
Breaks_btwn_Lecture    0
Study_area_rating      0
Same_section           0
dtype: int64

In [92]:
# Filling all the missing values in below column with 'None'
df2['Events_suggestions']=df2['Events_suggestions'].fillna('None')

In [93]:
df2

,Intake,Semester,Services_comm,Services_list,Commute,Additional_events,Events_suggestions,Breaks_btwn_Lecture,Study_area_rating,Same_section
0,Winter 2022,4th Semester,No,Commute/Transportation,No,Yes,,No,Yes,No
1,Winter 2022,4th Semester,No,"Career Services, Food Bank, SRC Service Office",No,Yes,,No,2,No
2,Winter 2023,1st Semester,No,"Housing, Career Services, Food Bank, Commute/T...",No,Yes,,Yes,3,Yes
3,Winter 2022,4th Semester,No,"Housing, Career Services, Commute/Transportati...",Yes,Yes,Food Services should be more flexible,Yes,4,No
4,Fall 2022,2nd Semester,Yes,"Career Services, Food Bank, Tax Assistance, Sa...",Yes,Yes,Plz organise most of the events in downtown,Yes,2,Yes
...,...,...,...,...,...,...,...,...,...,...
131,Winter 2022,4th Semester,Yes,"Career Services, Commute/Transportation, Print...",Yes,Yes,,Yes,4,Yes
132,Winter 2022,4th Semester,No,"Career Services, Food Bank, Commute/Transporta...",Yes,No,,No,5,Yes
133,Spring 2022,3rd Semester,Yes,"Career Services, Food Bank, Tax Assistance, Sa...",Yes,Yes,,Yes,5,Yes
134,Fall 2021,4th Semester,No,"Food Bank, Tax Assistance",No,Yes,Good infrastructure but less utilized,No,2,Yes


In [94]:
df2['Services_list'] = df2['Services_list'].str.strip()
df2['Services_list'] = df2['Services_list'].str.replace(' ', '')
one_hot_encoded = df2['Services_list'].str.strip().str.get_dummies(sep=',')
df2 = pd.concat([df2, one_hot_encoded], axis=1)
df2 = df2.drop("Services_list", axis=1)
df2.rename(columns={'Commute/Transportation': 'Commute_Transportation'}, inplace=True)


In [95]:
one_hot_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   CareerServices          136 non-null    int64
 1   Commute/Transportation  136 non-null    int64
 2   FoodBank                136 non-null    int64
 3   HealthandNursing        136 non-null    int64
 4   Housing                 136 non-null    int64
 5   ParkingServices         136 non-null    int64
 6   PrintingServices        136 non-null    int64
 7   SRCServiceOffice        136 non-null    int64
 8   Safewalk                136 non-null    int64
 9   TaxAssistance           136 non-null    int64
dtypes: int64(10)
memory usage: 10.8 KB


In [96]:
df2

,Intake,Semester,Services_comm,Commute,Additional_events,Events_suggestions,Breaks_btwn_Lecture,Study_area_rating,Same_section,CareerServices,Commute_Transportation,FoodBank,HealthandNursing,Housing,ParkingServices,PrintingServices,SRCServiceOffice,Safewalk,TaxAssistance
0,Winter 2022,4th Semester,No,No,Yes,,No,Yes,No,0,1,0,0,0,0,0,0,0,0
1,Winter 2022,4th Semester,No,No,Yes,,No,2,No,1,0,1,0,0,0,0,1,0,0
2,Winter 2023,1st Semester,No,No,Yes,,Yes,3,Yes,1,1,1,0,1,0,0,0,0,0
3,Winter 2022,4th Semester,No,Yes,Yes,Food Services should be more flexible,Yes,4,No,1,1,0,0,1,0,1,0,0,1
4,Fall 2022,2nd Semester,Yes,Yes,Yes,Plz organise most of the events in downtown,Yes,2,Yes,1,0,1,1,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Winter 2022,4th Semester,Yes,Yes,Yes,,Yes,4,Yes,1,1,0,0,0,0,1,0,0,0
132,Winter 2022,4th Semester,No,Yes,No,,No,5,Yes,1,1,1,1,0,0,0,0,1,1
133,Spring 2022,3rd Semester,Yes,Yes,Yes,,Yes,5,Yes,1,0,1,0,0,0,1,0,1,1
134,Fall 2021,4th Semester,No,No,Yes,Good infrastructure but less utilized,No,2,Yes,0,0,1,0,0,0,0,0,0,1


In [97]:
new_cols = []
for col in df2.columns:
    new_cols.append(col.replace(' ', '_'))
df2 = df2.rename(columns=dict(zip(df2.columns, new_cols)))
df2

,Intake,Semester,Services_comm,Commute,Additional_events,Events_suggestions,Breaks_btwn_Lecture,Study_area_rating,Same_section,CareerServices,Commute_Transportation,FoodBank,HealthandNursing,Housing,ParkingServices,PrintingServices,SRCServiceOffice,Safewalk,TaxAssistance
0,Winter 2022,4th Semester,No,No,Yes,,No,Yes,No,0,1,0,0,0,0,0,0,0,0
1,Winter 2022,4th Semester,No,No,Yes,,No,2,No,1,0,1,0,0,0,0,1,0,0
2,Winter 2023,1st Semester,No,No,Yes,,Yes,3,Yes,1,1,1,0,1,0,0,0,0,0
3,Winter 2022,4th Semester,No,Yes,Yes,Food Services should be more flexible,Yes,4,No,1,1,0,0,1,0,1,0,0,1
4,Fall 2022,2nd Semester,Yes,Yes,Yes,Plz organise most of the events in downtown,Yes,2,Yes,1,0,1,1,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Winter 2022,4th Semester,Yes,Yes,Yes,,Yes,4,Yes,1,1,0,0,0,0,1,0,0,0
132,Winter 2022,4th Semester,No,Yes,No,,No,5,Yes,1,1,1,1,0,0,0,0,1,1
133,Spring 2022,3rd Semester,Yes,Yes,Yes,,Yes,5,Yes,1,0,1,0,0,0,1,0,1,1
134,Fall 2021,4th Semester,No,No,Yes,Good infrastructure but less utilized,No,2,Yes,0,0,1,0,0,0,0,0,0,1


In [98]:
len(df2)

136

## Big-Query Table
### API Connection to Google BigQuery Database (surveyproject-378222.Capstone_Project)



In [99]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Intake                  136 non-null    object
 1   Semester                136 non-null    object
 2   Services_comm           136 non-null    object
 3   Commute                 136 non-null    object
 4   Additional_events       136 non-null    object
 5   Events_suggestions      136 non-null    object
 6   Breaks_btwn_Lecture     136 non-null    object
 7   Study_area_rating       136 non-null    object
 8   Same_section            136 non-null    object
 9   CareerServices          136 non-null    int64 
 10  Commute_Transportation  136 non-null    int64 
 11  FoodBank                136 non-null    int64 
 12  HealthandNursing        136 non-null    int64 
 13  Housing                 136 non-null    int64 
 14  ParkingServices         136 non-null    int64 
 15  Printi

In [100]:
#Connecting Big-Query Project 
client = bigquery.Client(project='surveyproject-378222')
sc = """
DROP TABLE IF EXISTS `surveyproject-378222.Capstone_Project.Survey2_Base_Table`;
"""
query_job = client.query(sc)
table_id = 'surveyproject-378222.Capstone_Project.Survey2_Base_Table'

# Defining the schema for the new table
schema = [
    bigquery.SchemaField('Intake', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('Semester', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('Services_comm', 'STRING'),
    bigquery.SchemaField('Services_list', 'STRING'),
    bigquery.SchemaField('Commute', 'STRING'),
    bigquery.SchemaField('Additional_events', 'STRING'),
    bigquery.SchemaField('Events_suggestions', 'STRING'),
    bigquery.SchemaField('Breaks_btwn_Lecture', 'BOOLEAN'),
    bigquery.SchemaField('Study_area_rating', 'STRING'),
    bigquery.SchemaField('Same_section', 'STRING'),
    bigquery.SchemaField('Career Services', 'STRING'),
    bigquery.SchemaField('Commute_Transportation', 'STRING'),
    bigquery.SchemaField('Food Bank', 'STRING'),
    bigquery.SchemaField('Health and Nursing', 'STRING'),
    bigquery.SchemaField('Housing', 'STRING'),
    bigquery.SchemaField('Parking Services', 'STRING'),
    bigquery.SchemaField('Printing Services', 'STRING'),
    bigquery.SchemaField('SRC Service Office', 'STRING'),
    bigquery.SchemaField('Safe walk', 'STRING'),
    bigquery.SchemaField('Tax Assistance', 'STRING')

]

try:
    # Try to get the existing table
    table = client.get_table(table_id)
    print(f'Table {table.table_id} already exists in BigQuery.')
except google.api_core.exceptions.NotFound:
    # If the table does not exist, create it
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created in BigQuery.')


project_id = 'surveyproject-378222'

client = bigquery.Client(project=project_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_dataframe(df2, table_id, job_config=job_config)
job.result()



Table Survey2_Base_Table created in BigQuery.


LoadJob<project=surveyproject-378222, location=northamerica-northeast2, id=6fb93287-61f2-445e-978b-1c996de1759f>

In [105]:
#Reading data from BigQuery to view the push data

Survey2_Base_Table= pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Survey2_Base_Table`',project_id='surveyproject-378222', dialect='standard') 
Survey2_Base_Table.describe()

Downloading: 100%|██████████|


,CareerServices,Commute_Transportation,FoodBank,HealthandNursing,Housing,ParkingServices,PrintingServices,SRCServiceOffice,Safewalk,TaxAssistance
count,136.0,136.0,136.0,136.0,136.0,136.0,136.0,136.0,136.0,136.0
mean,0.610294,0.257353,0.485294,0.308824,0.301471,0.227941,0.286765,0.294118,0.161765,0.441176
std,0.489486,0.438792,0.501631,0.463716,0.460593,0.421055,0.453923,0.45733,0.369596,0.498363
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [102]:
Survey2_Base_Table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Intake                  136 non-null    object
 1   Semester                136 non-null    object
 2   Services_comm           136 non-null    object
 3   Commute                 136 non-null    object
 4   Additional_events       136 non-null    object
 5   Events_suggestions      42 non-null     object
 6   Breaks_btwn_Lecture     136 non-null    object
 7   Study_area_rating       136 non-null    object
 8   Same_section            136 non-null    object
 9   CareerServices          136 non-null    Int64 
 10  Commute_Transportation  136 non-null    Int64 
 11  FoodBank                136 non-null    Int64 
 12  HealthandNursing        136 non-null    Int64 
 13  Housing                 136 non-null    Int64 
 14  ParkingServices         136 non-null    Int64 
 15  Printi

## Breaking Surevy2_Base_Table to 2 other Tables

In [104]:
 #Defining the SQL code to create tables and insert data
sql_code = """
DROP TABLE IF EXISTS `Capstone_Project.Survey_2`;
CREATE TABLE `Capstone_Project.Survey_2` (
  intake STRING,
  semester STRING,
  Services_comm STRING,
  Commute STRING,
  Additional_events STRING,
  Events_suggestions STRING,
  Breaks_btwn_Lecture STRING,
  Study_area_rating STRING,
  Same_section  STRING                 
);

INSERT INTO `surveyproject-378222.Capstone_Project.Survey_2`
SELECT intake, semester, Services_comm, Commute, Additional_events, 
       Events_suggestions, Breaks_btwn_Lecture, Study_area_rating, Same_section
FROM `surveyproject-378222.Capstone_Project.Survey2_Base_Table`;

DROP TABLE IF EXISTS `Capstone_Project.Survey2_services`;
CREATE TABLE `Capstone_Project.Survey2_services` (
  CareerServices INTEGER,
  Commute_Transportation INTEGER,
  FoodBank INTEGER,
  HealthandNursing INTEGER,
  Housing INTEGER,
  ParkingServices INTEGER,
  PrintingServices INTEGER,
  SRCServiceOffice INTEGER,
  Safewalk INTEGER,
  TaxAssistance  INTEGER        
);

INSERT INTO `surveyproject-378222.Capstone_Project.Survey2_services`
SELECT CareerServices, Commute_Transportation, FoodBank,
       HealthandNursing, Housing, ParkingServices,
       PrintingServices, SRCServiceOffice, Safewalk,
       TaxAssistance       
FROM `surveyproject-378222.Capstone_Project.Survey2_Base_Table`;
"""
query_job = client.query(sql_code)
query_job.result()


In [107]:
#reading Survey_2 table from google BigQuery 
Survey_2= pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Survey_2`',project_id='surveyproject-378222', dialect='standard')
Survey_2

Downloading: 100%|██████████|


,intake,semester,Services_comm,Commute,Additional_events,Events_suggestions,Breaks_btwn_Lecture,Study_area_rating,Same_section
0,Fall 2021,4th Semester,No,No,No,None,No,3,No
1,Fall 2021,4th Semester,No,No,Yes,There should be a two three day reading week k...,No,1,Yes
2,Fall 2021,4th Semester,No,No,Yes,There is a desperate need of career focused ev...,Yes,1,No
3,Fall 2021,4th Semester,No,No,Yes,COLLEGE SHOULD PROVIDE SOME MORE TIME IN THE ...,Yes,1,Yes
4,Fall 2021,4th Semester,No,No,Yes,This program is an example of diversity went w...,Yes,2,Yes
...,...,...,...,...,...,...,...,...,...
131,Winter 2023,1st Semester,Yes,No,Yes,None,Yes,3,No
132,Winter 2023,1st Semester,Yes,Yes,Yes,None,Yes,4,No
133,Winter 2023,1st Semester,Yes,Yes,Yes,It would be great to have more career and netw...,Yes,4,Yes
134,Winter 2023,1st Semester,Yes,Yes,Yes,None,Yes,4,Yes


In [106]:
#reading Survey2_services table from google BigQuery 
Survey2_services= pandas_gbq.read_gbq(f'SELECT * FROM `surveyproject-378222.Capstone_Project.Survey2_services`',project_id='surveyproject-378222', dialect='standard')
Survey2_services

Downloading: 100%|██████████|


,CareerServices,Commute_Transportation,FoodBank,HealthandNursing,Housing,ParkingServices,PrintingServices,SRCServiceOffice,Safewalk,TaxAssistance
0,1,0,1,1,0,1,1,1,0,0
1,1,0,1,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
131,1,1,1,1,1,0,1,1,1,1
132,1,0,1,0,1,0,0,0,0,1
133,1,1,0,0,1,0,0,0,0,1
134,1,1,1,1,1,0,0,0,0,1
